In [1]:
# There, done. That was easy.
# That's all we needed to do, right?
import numpy as np
import pandas as pd
import xgboost as xgb

In [5]:
# Different years have VERY different schemas, so preprocessing is gonna suck.
# Plus all the one-hot variables.
# We'll do it later.
data = [pd.read_csv('data/201%d.csv' % i) for i in xrange(1, 8)]